# Installs

In [0]:
%pip install unitycatalog-ai[databricks] unitycatalog-langchain[databricks] langgraph==0.2.74 databricks-langchain==0.3.0

In [0]:
dbutils.library.restartPython()

# Setup

In [0]:
from databricks_langchain import ChatDatabricks

# Create Some Basic Tools

In [0]:
import math
from langchain_core.tools import tool

@tool
def solve_quadratic(a, b, c):
    """
    Solves the quadratic formula give the inputs a, b, and c.
    Returns the number of real roots, plus their values if found.
    """
    # Calculate the discriminant
    discriminant = b**2 - 4*a*c
    
    # Check if the discriminant is negative, zero, or positive
    if discriminant < 0:
        return "No real roots"
    elif discriminant == 0:
        x = -b / (2*a)
        return f"One real root: {x}"
    else:
        x1 = (-b + math.sqrt(discriminant)) / (2*a)
        x2 = (-b - math.sqrt(discriminant)) / (2*a)
        return f"Two real roots: {x1} and {x2}"
    
@tool
def pythagorean_theorem(a=None, b=None, c=None):
    """
    Solves the Pythagorean Theorem formula give the inputs a, b, and c.
    Returns the length of the third side. Returns an error unless exactly one side length is None.
    """
    if a is None:
        return math.sqrt(c**2 - b**2)
    elif b is None:
        return math.sqrt(c**2 - a**2)
    elif c is None:
        return math.sqrt(a**2 + b**2)
    else:
        raise("Please provide exactly one side length as None")


In [0]:
print(f"Name: {solve_quadratic.name}\n")
print(f"Description: {solve_quadratic.description}\n")
print(f"Args: {solve_quadratic.args}")

# Create LLM + ReAct Agent

In [0]:
llm = ChatDatabricks(
    endpoint= "gpt-4o"
)

In [0]:
prompt = "You are an AI agent who loves to help students with their maths homework!"

In [0]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, [solve_quadratic, pythagorean_theorem], prompt=prompt)

# Use The Tools!

In [0]:
result = agent_executor.invoke({"messages": [("user", "How many real roots are there for a quadratic equation with a=1, b=2, and c=3?")]})

In [0]:
result = agent_executor.invoke({"messages": [("user", "How long is the third side of a right triangle with a=3 and b=4?")]})

In [0]:
result = agent_executor.invoke(
    {
        "messages": [
            ("user", "How long is the third side of a right triangle with a=3 and b=4?"),
            ("user", "How many roots does the quadratic curve have where a=2, c=2, and b equals the answer to the previous question?"),
        ]
    }
)

#... save them to Unity Catalog!

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
client = DatabricksFunctionClient()

In [0]:
def solve_quadratic(a: float, b: float, c: float) -> str:
    """
    Solves the quadratic formula give the inputs a, b, and c.
    Returns the number of real roots, plus their values if found.

    Args:
        a (float): The first coefficient of the quadratic equation.
        b (float): The second coefficient of the quadratic equation.
        c (float): The third coefficient of the quadratic equation.

    Returns:
        str: The number of real roots, plus their values if found.
    """
    # Calculate the discriminant
    discriminant = b**2 - 4*a*c
    
    # Check if the discriminant is negative, zero, or positive
    if discriminant < 0:
        return "No real roots"
    elif discriminant == 0:
        x = -b / (2*a)
        return f"One real root: {x}"
    else:
        x1 = (-b + math.sqrt(discriminant)) / (2*a)
        x2 = (-b - math.sqrt(discriminant)) / (2*a)
        return f"Two real roots: {x1} and {x2}"

In [0]:
CATALOG = "advancing_ai"
SCHEMA = "agentic-tools"

function_info = client.create_python_function(
  func= solve_quadratic,
  catalog= CATALOG,
  schema= SCHEMA,
  replace= True
)